##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Train sentiment analysis models with TensorFlow Lite Model Maker

In this step, we will use the [Stanford Sentiment Treebank v2](https://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf)(SST-2) dataset to train the model. The dataset contains more than 11,000 sentences from movie reviews and the sentiment (positive or negative) of each sentence. Here is some examples from the dataset:

| Sentence | Sentiment |
|-|-|
| This is a film well worth seeing , talking and singing heads and all . | Positive |
| What really surprises about Wisegirls is its low-key quality and genuine tenderness . | Positive |
| Though everything might be literate and smart , it never took off and always seemed static . | Negative |

We will use TensorFlow Lite Model Maker to train **text classification** models with this dataset. **Sentiment analysis** is one type of **text classification** because its goal is to classify a body of text into positive or negative sentiment. We will train two models:
* An `average word vector` model: This is a light-weight model which is less 1MB and also runs very fast.
* A `MobileBERT` model: This is a more accurate but larger model.

You also can choose a model architecture in advance and train only one model. However, it is recommended to train multiple models with different architecture and examine the trade-off between model size and accuracy and see which model is more suitable for your use case.

*Note: Before you proceed, make sure that the Colab notebook is in GPU mode to speed up model training.*

### Install the required packages

Let's start by installing TFLite Model Maker and import the required packages.

In [ ]:
!pip install -q tflite-model-maker

     |████████████████████████████████| 112kB 2.9MB/s 
     |████████████████████████████████| 1.1MB 9.0MB/s 
     |████████████████████████████████| 849kB 17.2MB/s 
     |████████████████████████████████| 81kB 9.0MB/s 
     |████████████████████████████████| 1.0MB 20.6MB/s 
     |████████████████████████████████| 325.1MB 50kB/s 
     |████████████████████████████████| 174kB 53.1MB/s 
     |████████████████████████████████| 358kB 53.3MB/s 
     |████████████████████████████████| 102kB 12.9MB/s 
     |████████████████████████████████| 36.7MB 65kB/s 
     |████████████████████████████████| 296kB 50.6MB/s 
     |████████████████████████████████| 471kB 51.1MB/s 
     |████████████████████████████████| 10.6MB 175kB/s 


In [ ]:
import numpy as np
import os

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker import TextClassifierDataLoader

### Download the SST-2 dataset

In [ ]:
data_dir = tf.keras.utils.get_file(
      fname='SST-2.zip',
      origin='https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSST-2.zip?alt=media&token=aabc5f6b-e466-44a2-b9b4-cf6337f84ac8',
      extract=True)
data_dir = os.path.join(os.path.dirname(data_dir), 'SST-2')

7446528/7439277 [==============================] - 0s 0us/step


## Train our first model: an `Average Word Vector` model

This is the light-weight model architecture that can be trained in a few minutes. The model is small and runs fast with decent accuracy.

### Specify the model architecture

In [ ]:
awv_spec = model_spec.get('average_word_vec')

### Pre-process data
Convert the SST-2 dataset to input format that is required by TFLite Model Maker. Please note that there are two portion of our dataset:
* `train.tsv`: The **training** dataset that the model will learn from.
* `dev.tsv`: The **evaluation** dataset that the model doesn't see when it is trained. This is used to test if the sentiment analysis is able to generalize well on new data that it has never seen before.

In [ ]:
awv_train_data = TextClassifierDataLoader.from_csv(
      filename=os.path.join(os.path.join(data_dir, 'train.tsv')),
      text_column='sentence',
      label_column='label',
      model_spec=awv_spec,
      delimiter='\t',
      is_training=True)
awv_test_data = TextClassifierDataLoader.from_csv(
      filename=os.path.join(os.path.join(data_dir, 'dev.tsv')),
      text_column='sentence',
      label_column='label',
      model_spec=awv_spec,
      delimiter='\t',
      is_training=False)

INFO:tensorflow:Saved vocabulary in /tmp/tmp5d0k8tzt/e72d242a17446c5dc91aa41e181ce914_vocab.


### Train the model
We use TFLite Model Maker to train a TensorFlow model with the training dataset.

In [ ]:
awv_model = text_classifier.create(awv_train_data, model_spec=awv_spec, epochs=10)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Epoch 1/10
2104/2104 [==============================] - 12s 6ms/step - loss: 0.6846 - accuracy: 0.5565
Epoch 2/10
2104/2104 [==============================] - 10s 5ms/step - loss: 0.5802 - accuracy: 0.7036
Epoch 3/10
2104/2104 [==============================] - 10s 5ms/step - loss: 0.4430 - accuracy: 0.7988
Epoch 4/10
2104/2104 [==============================] - 10s 5ms/step - loss: 0.3955 - accuracy: 0.8281
Epoch 5/10
2104/2104 [==============================] - 10s 5ms/step - loss: 0.3711 - accuracy: 0.8411
Epoch 6/10
2104/2104 [==============================] - 10s 5ms/step - loss: 0.3557 - accuracy: 0.8495
Epoch 7/10
2104/2104 [==============================] - 10s 5ms/step - loss: 0.3434 - accuracy: 0.8576
Epoch 8/10
2104/2104 [==============================] - 10s 5ms/step - loss: 0.3357 - accuracy: 0.8605
Epoch 9/10
2104/2104 [==============================] - 10s 5ms/step - loss: 0.3288 - accuracy: 0.8629
Epoch 10/10
2104/2104 [==============================] - 10s 5ms/step - l

### Evaluate the model

Let's see how the model performs against new data in the evaluation dataset that it has never seen before.

In [ ]:
awv_model.evaluate(awv_test_data)

28/28 [==============================] - 0s 9ms/step - loss: 0.5168 - accuracy: 0.8303


[0.5167555809020996, 0.8302752375602722]

### Export the model into TensorFlow Lite format

In [ ]:
awv_model.export(export_dir='average_word_vec/')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: /tmp/tmpz9sm8akr/assets


...

INFO:tensorflow:Saved vocabulary in /tmp/tmphl5pl6r1/vocab.txt.


INFO:tensorflow:Saving labels in /tmp/tmphl5pl6r1/labels.txt.


INFO:tensorflow:Saving labels in /tmp/tmphl5pl6r1/labels.txt.


Finished populating metadata and associated file to the model:
average_word_vec/model.tflite
The metadata json file has been saved to:
average_word_vec/model.json
The associated file that has been been packed to the model is:
['vocab.txt', 'labels.txt']


/usr/local/lib/python3.6/dist-packages/tensorflow_lite_support/metadata/metadata.py:344: UserWarning: File, 'vocab.txt', does not exsit in the metadata. But packing it to tflite model is still allowed.
  "tflite model is still allowed.".format(f))


## Optional: Train our second model: an `MobileBERT` model

`MobileBERT` models are more accurate than `Average Word Vector` models but they are also larger. It will take about 30 minutes to be train our `MobileBERT` sentiment analysis model.

### Specify the model architecture

You will see that all steps below are mostly the same as training a `Average Word Vector` sentiment analysis model. Here we use `MobileBert` model specification instead of the `Average Word Vector` specification.

In [ ]:
mb_spec = model_spec.get('mobilebert_classifier')

### Pre-process data
Convert the SST-2 dataset to input format that is required by TFLite Model Maker. As `MobileBERT` models and `Average Word Vector` models take different input format, we'll need to run the data preprocess step again.

In [ ]:
mb_train_data = TextClassifierDataLoader.from_csv(
      filename=os.path.join(os.path.join(data_dir, 'train.tsv')),
      text_column='sentence',
      label_column='label',
      model_spec=mb_spec,
      delimiter='\t',
      is_training=True)
mb_test_data = TextClassifierDataLoader.from_csv(
      filename=os.path.join(os.path.join(data_dir, 'dev.tsv')),
      text_column='sentence',
      label_column='label',
      model_spec=mb_spec,
      delimiter='\t',
      is_training=False)

### Train the model
We use TFLite Model Maker to train a TensorFlow model with the training dataset.

In [ ]:
mb_model = text_classifier.create(mb_train_data, model_spec=mb_spec, epochs=3)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Epoch 1/2
   1/1403 [..............................] - ETA: 18:19:26 - loss: 6.2104 - test_accuracy: 0.5417WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1371: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


Instructions for updating:
use `tf.profiler.experimental.stop` instead.


   2/1403 [..............................] - ETA: 9:25:17 - loss: 6.6119 - test_accuracy: 0.5312 WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.4344s vs `on_train_batch_end` time: 0.9315s). Check your callbacks.


1403/1403 [==============================] - 640s 456ms/step - loss: 1.1017 - test_accuracy: 0.6984
Epoch 2/2
1403/1403 [==============================] - 594s 423ms/step - loss: 0.2188 - test_accuracy: 0.9131


### Evaluate the model

Let's see how the model performs against new data in the evaluation dataset that it has never seen before.

In [ ]:
mb_model.evaluate(mb_test_data)

28/28 [==============================] - 7s 255ms/step - loss: 0.2671 - test_accuracy: 0.9002


[0.26705214381217957, 0.9002293348312378]

### Export the model into TensorFlow Lite format

There is a slight different between the export code for `MobileBERT` compared to the `Average Word Vector`. As `MobileBERT` model size is large, we apply [quantization](https://www.tensorflow.org/lite/performance/post_training_quantization) when exporting to TFLite format to reduce the model size by about 4 times.

In [ ]:
config = configs.QuantizationConfig.create_dynamic_range_quantization(optimizations=[tf.lite.Optimize.DEFAULT])
config._experimental_new_quantizer = True

mb_model.export(export_dir='mobilebert/', quantization_config=config)

INFO:tensorflow:Assets written to: /tmp/tmpl28ytd6b/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmpl28ytd6b/saved_model/assets


INFO:tensorflow:Vocab file and label file are inside the TFLite model with metadata.


INFO:tensorflow:Vocab file and label file are inside the TFLite model with metadata.


INFO:tensorflow:Saved vocabulary in /tmp/tmpigxjviyy/vocab.txt.


INFO:tensorflow:Saved vocabulary in /tmp/tmpigxjviyy/vocab.txt.


INFO:tensorflow:Saving labels in /tmp/tmpigxjviyy/labels.txt.


INFO:tensorflow:Saving labels in /tmp/tmpigxjviyy/labels.txt.


Finished populating metadata and associated file to the model:
mobilebert/model.tflite
The metadata json file has been saved to:
mobilebert/model.json
The associated file that has been been packed to the model is:
['labels.txt', 'vocab.txt']


## Optional: Compare the two models

Let's see the model size:

In [ ]:
print('Average Word Vector')
!ls -all --block-size=KB average_word_vec/*.tflite
print('MobileBERT')
!ls -all --block-size=KB mobilebert/*.tflite

Average Word Vector
-rw-r--r-- 1 root root 776kB Oct  2 05:26 average_word_vec/model.tflite
MobileBERT
-rw-r--r-- 1 root root 25708kB Oct  2 05:49 mobilebert/model.tflite


You can optionally use the [TFLite benchmark tool](https://www.tensorflow.org/lite/performance/measurement#benchmark_tools) to measure the each model speed on mobile phones. The model speed of a certain model architecture does not change no matter which dataset you use to train the model. Therefore, you can refer to the model speed data on TensorFlow Lite website without running the benchmark by yourself.

Let's compare our two sentiment analysis models:

| Model Architecture | Accuracy | Size | Speed |
|-|-|-|-|
| Average Word Vector | 83% | 0.77MB | 0.05ms |
| MobileBERT | 90% | 25.7MB | 83ms |

*Note: Model speed was measured on Pixel 4 CPU, running on 2 threads.*

## Download the TFLite model.

We want to start small with a light-weight model, so the `Average Word Vector` model seems to be more suitable for our use case. Let's download it.

In [ ]:
!cp average_word_vec/model.tflite sentiment_analysis.tflite

from google.colab import files
files.download('sentiment_analysis.tflite') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Now you have finished training the two TFLite sentiment analysis models. Let's go back to the next step in the codelab.